In [1]:
### Find Missing Plants

In [2]:
import sys, os
sys.path.append('/Users/zaneselvans/code/catalyst/pudl')
sys.path.append('/Users/christinagosnell/code/pudl')
sys.path.append('/Users/Nannerz/Desktop/working/pudl/')
sys.path.append('/home/alana/Dropbox/catalyst/pudl/')
sys.path.append(os.path.abspath(os.path.join('..','..')))
from pudl import ferc1, pudl, models, models_ferc1, settings, constants, eia923, models_eia923
from pudl.eia923 import get_eia923_page
from pudl.eia923 import get_eia923_xlsx, get_eia923_column_map
from pudl.eia923 import get_eia923_plants
from pudl.eia860 import get_eia860_xlsx
from pudl.pudl import *
from pudl.constants import tab_map_eia923
import numpy as np
import pandas as pd
from pandas import DataFrame
import csv
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
from sqlalchemy import Integer, String, Numeric, Boolean, Float

In [3]:
%load_ext autoreload
%autoreload 1
%aimport pudl.pudl
%aimport pudl.ferc1
%aimport pudl.eia923
%aimport pudl.constants
%aimport pudl.settings
%aimport pudl.models
%aimport pudl.models_eia923

In [4]:
eia923_years=[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

In [ ]:
eia923_xlsx = get_eia923_xlsx(eia923_years)

In [7]:
eia923_plants = get_eia923_plants(eia923_years, eia923_xlsx)
eia923_plant_ids = eia923_plants[['plant_id',]]

Converting EIA 923 plant_frame to DataFrame...
Converting EIA 923 generation_fuel to DataFrame...
Converting EIA 923 boiler_fuel to DataFrame...
Converting EIA 923 generator to DataFrame...
Converting EIA 923 fuel_receipts_costs to DataFrame...


In [ ]:
output_tab_plants = pd.read_excel('../id_mapping/mapping_eia923_ferc1.xlsx', 'plants_output', na_values='', keep_default_na=False,
                              converters={'plant_id': int,
                                          'plant_name': str,
                                          'respondent_id_ferc': int,
                                          'respondent_name_ferc': str,
                                          'plant_name_ferc': str,
                                          'plant_id_eia': int,
                                          'plant_name_eia': str,
                                          'operator_name_eia': str,
                                          'operator_id_eia': int})
output_plant_ids = output_tab_plants[['plant_id_eia']]
output_plant_ids = output_plant_ids.dropna()

In [ ]:
eia923_plant_ids_tuple = set([ tuple(line) for line in eia923_plant_ids.values.tolist()])
output_plant_ids_tuple = set([ tuple(line) for line in output_plant_ids.values.tolist()])
missing_plant_ids = pd.DataFrame(list(eia923_plant_ids_tuple.difference(output_plant_ids_tuple)))
missing_plant_ids = missing_plant_ids.rename(columns={0: 'plant_id',})

In [ ]:
missing_plants = missing_plant_ids.merge(eia923_plants[['plant_id','plant_name','operator_name','operator_id']],  on='plant_id', how='left')

In [ ]:
# missing_plants.to_csv('../id_mapping/missing_plants.csv', index=False)

## Generate a list of EIA 860 ids from a new year's data.

In order for the create_dfs_eia860 function to work, we need to include 2012 data. We'll also want to include the year we're trying to ingest!

In [3]:
eia860_years=[2012, 2016]

We'll use create_dfs_eia860 to create a dictionary of table names and corresponding DataFrames made up of the data from each EIA 860 table.

In [4]:
dfs_eia860 = eia860.create_dfs_eia860(eia860_years=eia860_years)

Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2016.
Converting EIA 860 boiler_generator_assn to DataFrame...
Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2016.
Converting EIA 860 utility to DataFrame...
Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2016.
Converting EIA 860 plant to DataFrame...
Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2016.
Converting EIA 860 generator_existing to DataFrame...
Converting EIA 860 generator_proposed to DataFrame...
Converting EIA 860 generator_retired to DataFrame...
Reading EIA 860 spreadsheet data for 2012.
Reading EIA 860 spreadsheet data for 2016.
Converting EIA 860 ownership to DataFrame...


We'll then use the keys to produce DataFrames representing the data in each table.

In [5]:
dfs_eia860.keys()

dict_keys(['boiler_generator_assn', 'utility', 'plant', 'generator_existing', 'generator_proposed', 'generator_retired', 'ownership'])

In [6]:
bga = dfs_eia860['boiler_generator_assn']
u_df = dfs_eia860['utility']
p_df = dfs_eia860['plant']
ge_df = dfs_eia860['generator_existing']
gp_df = dfs_eia860['generator_proposed']
gr_df = dfs_eia860['generator_retired']
o_df = dfs_eia860['ownership']

We'll then extract every plant_id from each table and append them together into one large numpy array of all the plant_ids.

In [7]:
bga_plants = bga['plant_id']
p_plants = p_df['plant_id']
ge_plants = ge_df['plant_id']
gp_plants = gp_df['plant_id']
gr_plants = gr_df['plant_id']
o_plants = o_df['plant_id']

In [8]:
eia860_plant_ids = bga_plants.append([p_plants,ge_plants,gp_plants,gr_plants,o_plants]).unique()

We'll next create a DataFrame containing all of the plant_ids from the previous years' data and extract the plant_ids as another data Series.

In [10]:
output_tab_plants = pd.read_excel('../../id_mapping/mapping_eia923_ferc1.xlsx', 'plants_output', na_values='', keep_default_na=False,
                              converters={'plant_id': int,
                                          'plant_name': str,
                                          'respondent_id_ferc': int,
                                          'respondent_name_ferc': str,
                                          'plant_name_ferc': str,
                                          'plant_id_eia': int,
                                          'plant_name_eia': str,
                                          'operator_name_eia': str,
                                          'operator_id_eia': int})
output_plant_ids = output_tab_plants[['plant_id_eia']]
output_plant_ids = output_plant_ids.dropna()

In [11]:
output_plant_ids = output_plant_ids['plant_id_eia']

In order to compare these plant_ids with the numpy array of new plant_ids, we'll need to convert the data Series into a numpy array.

In [14]:
output_plant_ids_array = output_plant_ids.as_matrix()

We can then compare the two arrays and extract the new plant_ids.

In [15]:
difference = np.setdiff1d(eia860_plant_ids,output_plant_ids_array).astype(int)

In [16]:
difference_df = pd.DataFrame(difference,columns=['plant_id'])

We'll then create merge these plant_ids with data from the full EIA 860 plant table to produce an output containing not only the plant_id but the plant_name, operator_id, and operator_name.

In [17]:
eia_860_new_plant_info = difference_df.merge(p_df, how='left')

In [18]:
eia_860_plant_info_simple = eia_860_new_plant_info[['plant_id','plant_name','operator_name','operator_id']]

Then we'll create a csv in order to add the new plant_ids and associated information to our ID matching spreadsheet.

In [83]:
difference_csv = eia_860_plant_info_simple.to_csv('eia860_2016_plants.csv')